# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Final Project: Part 2 - Feature Extraction


In any practical machine learning problem, the data preparation and feature extraction stages are the most important and time-consuming. The final project exposes you to a real-world dataset. In this part of the final project, you are responsible to creating features that will be meaningful for prediction. Features are evaluated based on Information Gain, which you implemented in [Assignment 2](../assignment-2/assignment-2.ipynb).

Here is what will work well in this project:

* Extract some sample data, load it in [R](https://www.r-project.org), and do some intial analysis. Feel free to build models there to get a feel for the best features.
* Join the different tables--they are there for a reason. 
* Get creative.
* Read some of the Kaggle competition forums and kernels. 

Here is what will NOT work:

* Do not use only the features as provided in application_train.
* Do not try implementing new learning algorithm in order to generate features. If you find something that works, investigate what features were helpful and add the features. 
* Do not build lookup tables "embeddings" or other things you might have read about but were not covered in class. 
* Do not try to build a kernel matrix on all pairs. Re-evaluate the kernel instead.

In [40]:
require './assignment_lib'

false

In [41]:
dir = "/home/dataset"
$dev_db = SQLite3::Database.new "#{dir}/credit_risk_data_dev.db", results_as_hash: true, readonly: true

#<SQLite3::Database:0x00000000110b1718 @tracefunc=nil, @authorizer=nil, @encoding=nil, @busy_handler=nil, @collations={}, @functions={}, @results_as_hash=true, @type_translation=nil, @readonly=true>

## Question 1.1 (10 Points)

Implement ```create_dataset``` which runs an SQL query on a database and constructs a dataset like those we have used in this course. Add an ```id``` field for the ```SK_ID_CURR``` and store the ```TARGET``` in ```label```. 

If the query is:
```sql
select sk_id_curr, target, ext_source_1 from application_train  where ext_source_1 <> '' order by sk_id_curr;
```

then the result is:

```json
[
    {"label":1,"id":100002,"features":{"ext_source_1":0.08303696739132256}},
    {"label":0,"id":100015,"features":{"ext_source_1":0.7220444501416448}}
]
...
```
Note the features should not include the ID or Label. Feature keys should be lowercase and only contain keys fo which ```key.is_a? String``` returns true.



In [42]:
def create_dataset db, sql
  dataset = []
  db.execute sql do |row|
    # BEGIN YOUR CODE
    data = Hash.new
    data["features"] = Hash.new
    
    row.each do |k, v|
      if k == "TARGET"
        data["label"] = row["TARGET"]
      elsif k == "SK_ID_CURR"
        data["id"] = row["SK_ID_CURR"]
      elsif k.is_a? String
        data["features"][k.downcase] = row[k]
      end
    end
    dataset << data
    #END YOUR CODE
  end
  return dataset
end

:create_dataset

In [43]:
def test_11()
  dataset = create_dataset $dev_db, "select sk_id_curr, target, ext_source_1 from application_train where ext_source_1 <> '' 
order by sk_id_curr limit 37"
  assert_equal 37, dataset.size
  assert_true(dataset[0]["features"].has_key? "ext_source_1")
  assert_equal(1, dataset[0]["features"].size)
  assert_equal(100002, dataset[0]["id"])  
  assert_in_delta(0.08303696, dataset[0]["features"]["ext_source_1"], 1e-4)
  assert_equal(1, dataset[0]["label"])    
end

test_11()

## Question 1.1 (20 points)

Copy and revise **your** information gain calculation for numeric and categorical features, from [Assignment 2](../assignment-2/assignment-2.ipynb). Copy the following implementations

* Class Distribution
* Entropy
* Information Gain after splitting
* Information gain for numerical features (fast version)


In [44]:
def class_distribution dataset
  # BEGIN YOUR CODE
  output = Hash.new {|h,k| h[k] = 0}
  
  for data in dataset
    output[data["label"]] += 1
  end

  sum = 0.0
  
  output.each_value do |v|
    sum += v
  end
  
  output.each {|k,v| output[k] = v /sum}
  
  return output  
  #END YOUR CODE
end

:class_distribution

In [45]:
def entropy dist
  # BEGIN YOUR CODE
  cEnt = 0.0
  sum = 0.0
  
  dist.each_value do |v|
    sum += v
  end
  
  if sum == 0.0
    return 0
  end
  
  dist.each_value do |v|
    prob = v/sum
    if prob < 1e-5
      next
    else
      cEnt -= prob*Math.log(prob)
    end
  end
  
  return cEnt
  #END YOUR CODE
end

:entropy

In [46]:
def test_12_1()
  # Check that there are three classes
  dataset = create_dataset $dev_db, "select target, sk_id_curr, ext_source_1, flag_own_car from application_train where ext_source_1 <> ''"
  dist = class_distribution dataset
  h0 = entropy dist
  assert_in_delta(0.2686201883261589, h0, 1e-3)
end

test_12_1()

In [47]:
def information_gain h0, splits
  # BEGIN YOUR CODE
  #missing values operations
  splits.delete("")
  
  sum = 0
  ig = h0
  
  splits.each_value do |v|
    sum += v.length
  end
  
  splits.each_value do |v|
    ig -= (entropy(class_distribution(v))) * v.length / sum
  end
  return ig
  #END YOUR CODE
end

:information_gain

In [48]:
def test_12_2()
  # Check that there are three classes
  dataset = create_dataset $dev_db, "select target, sk_id_curr, ext_source_1, flag_own_car from application_train where ext_source_1 <> ''"
  dist = class_distribution dataset
  h0 = entropy dist
  
  splits = dataset.group_by {|row| row["features"]["flag_own_car"]}
  ig = information_gain h0, splits
  assert_in_delta(0.0002206258541794237, ig, 1e-4)
end

test_12_2()

In [49]:
def find_split_point_numeric dataset, h0, fname
  # BEGIN YOUR CODE
  ig_max = 0
  t_max = nil

  # Missing value operations
  non_missing_value_dataset = []
  dataset.each do |row|
    if row["features"][fname] != "" && row["features"][fname] != nil
      non_missing_value_dataset << row
    end
  end
  
  counts_right = Hash.new {|h,k| h[k] = 0}
  counts_left = Hash.new {|h,k| h[k] = 0}
  v_left = 0.0
  v_right = non_missing_value_dataset.size.to_f
  
  feature_groups = non_missing_value_dataset.group_by {|row| row["features"].fetch(fname, 0.0)}
  
  feature_groups.each_key do |key|
    count_temp = Hash.new {|h,k| h[k] = 0}  
    feature_groups[key].each do |row| 
      count_temp[row["label"]] += 1
      counts_right[row["label"]] += 1
    end
    feature_groups[key] = count_temp
  end
  
  threshold_group = feature_groups.keys.sort
  t = threshold_group.shift
  
  feature_groups[t].each_key do |k| 
    counts_left[k] += feature_groups[t][k]
    counts_right[k] -= feature_groups[t][k]
    v_left += feature_groups[t][k]
    v_right -= feature_groups[t][k]
  end
  
  threshold_group.each.with_index do |t, i|
    v_left_n = v_left / non_missing_value_dataset.size
    v_right_n = v_right / non_missing_value_dataset.size
    
    d_left = Hash.new
    d_right = Hash.new
    counts_left.each_key {|k| d_left[k] = counts_left[k] / v_left}
    counts_right.each_key {|k| d_right[k] = counts_right[k] / v_right}
        
    h_left = entropy(d_left)
    h_right = entropy(d_right)    
    ig = h0 - (v_left_n * h_left + v_right_n * h_right)
    if ig > ig_max
      ig_max = ig
      t_max = t
    end

    feature_groups[t].each_key do |k| 
      counts_left[k] += feature_groups[t][k]
      counts_right[k] -= feature_groups[t][k]
      v_left += feature_groups[t][k]
      v_right -= feature_groups[t][k]
    end
  end

  return [t_max, ig_max]
  #END YOUR CODE
end

:find_split_point_numeric

In [50]:
def test_12_3()
  # Check that there are three classes
  dataset = create_dataset $dev_db, "select target, sk_id_curr, ext_source_1, flag_own_car from application_train where ext_source_1 <> ''"
  dist = class_distribution dataset
  h0 = entropy dist
  
  t, ig = find_split_point_numeric dataset, h0, "ext_source_1"
  assert_in_delta(0.009751743140812785, ig, 1e-4)
end

test_12_3()

## Question 2.1 (70 Points)

Using whatever external software you want (hosted on your own devices), provide 15+ different features that have information >= 0.005. You may to implement several cells below, so please insert them above the test. 

Features must only be derived from the database but you are free to write whatever SQL queries you want. You may create temporary tables, but the database is read-only.

Pay close attention to the following aspects of feature design:

* Normalization: Z-score, L2, Min-Max, etc.
* Sparsity / missing values
* Frequency: Information is easily fooled by features with many values.
* Joins: Some of the best features in this dataset combine two columns from different tables.
* Transformations: One-hot, Binning, Discretization, Non-linear transformation

In [51]:
def information_gain_categorical dataset, h0, fname
  splits = dataset.group_by {|row| row["features"][fname]}
  ig = information_gain h0, splits
  return ig
end

:information_gain_categorical

In [52]:
#Add extra cells as needed

In [53]:
def extract_features db
  dataset_temp = []
  # BEGIN YOUR CODE
  sql = "SELECT * FROM application_train"
  dataset = create_dataset db, sql
  h0 = entropy(class_distribution(dataset))
  selected_features = []
  
  dataset[0]["features"].each do |k, v|
    ig = 0
    if v.is_a? Numeric
      t, ig = find_split_point_numeric dataset, h0, k
    elsif v.is_a? String
      ig = information_gain_categorical dataset, h0, k
    end
    
    if ig > 0.005
      selected_features << k
    end
  end
  puts selected_features
  
  dataset.each do |row|
    data = Hash.new
    data["features"] = Hash.new
    data["id"] = row["id"]
    data["label"] = row["label"]
    
    for fname in selected_features
      data["features"][fname] = row["features"][fname]
    end
    dataset_temp << data
  end
  #END YOUR CODE
  return dataset_temp
end

:extract_features

In [54]:
extracted_dataset = extract_features($dev_db)
extracted_dataset[0]

["own_car_age", "ext_source_1", "ext_source_2", "ext_source_3", "apartments_avg", "basementarea_avg", "years_beginexpluatation_avg", "years_build_avg", "commonarea_avg", "elevators_avg", "entrances_avg", "floorsmax_avg", "floorsmin_avg", "landarea_avg", "livingapartments_avg", "livingarea_avg", "nonlivingapartments_avg", "nonlivingarea_avg", "apartments_mode", "basementarea_mode", "years_beginexpluatation_mode", "years_build_mode", "commonarea_mode", "elevators_mode", "entrances_mode", "floorsmax_mode", "floorsmin_mode", "landarea_mode", "livingapartments_mode", "livingarea_mode", "nonlivingapartments_mode", "nonlivingarea_mode", "apartments_medi", "basementarea_medi", "years_beginexpluatation_medi", "years_build_medi", "commonarea_medi", "elevators_medi", "entrances_medi", "floorsmax_medi", "floorsmin_medi", "landarea_medi", "livingapartments_medi", "livingarea_medi", "nonlivingapartments_medi", "nonlivingarea_medi", "fondkapremont_mode", "housetype_mode", "totalarea_mode", "wallsmate

{"features"=>{"own_car_age"=>"", "ext_source_1"=>0.08303696739132256, "ext_source_2"=>0.2629485927471776, "ext_source_3"=>0.13937578009978951, "apartments_avg"=>0.0247, "basementarea_avg"=>0.0369, "years_beginexpluatation_avg"=>0.9722, "years_build_avg"=>0.6192, "commonarea_avg"=>0.0143, "elevators_avg"=>0, "entrances_avg"=>0.069, "floorsmax_avg"=>0.0833, "floorsmin_avg"=>0.125, "landarea_avg"=>0.0369, "livingapartments_avg"=>0.0202, "livingarea_avg"=>0.019, "nonlivingapartments_avg"=>0, "nonlivingarea_avg"=>0, "apartments_mode"=>0.0252, "basementarea_mode"=>0.0383, "years_beginexpluatation_mode"=>0.9722, "years_build_mode"=>0.6341, "commonarea_mode"=>0.0144, "elevators_mode"=>0, "entrances_mode"=>0.069, "floorsmax_mode"=>0.0833, "floorsmin_mode"=>0.125, "landarea_mode"=>0.0377, "livingapartments_mode"=>0.022, "livingarea_mode"=>0.0198, "nonlivingapartments_mode"=>0, "nonlivingarea_mode"=>0, "apartments_medi"=>0.025, "basementarea_medi"=>0.0369, "years_beginexpluatation_medi"=>0.9722, 

In [55]:
assert_not_nil extracted_dataset
assert_equal 15334, extracted_dataset.size
assert_true(extracted_dataset.all? {|row| row["features"].size >= 8}, "At least 6 non-zero features per row")
assert_true(extracted_dataset.flat_map {|row| row["features"].keys}.uniq.size >= 15,  "At least 15 features")

In [56]:
assert_equal 15334, extracted_dataset.collect {|row| row["id"]}.uniq.size
assert_equal 2, extracted_dataset.collect {|row| row["label"]}.uniq.size

h0 = entropy(class_distribution(extracted_dataset))
assert_in_delta(0.2797684909805576, h0, 1e-3)

In [57]:
features = extracted_dataset.flat_map {|row| row["features"].keys}.uniq
numeric_features = features.select {|k| extracted_dataset.reject {|row| row["features"][k] == ""}.all? {|row| row["features"].fetch(k, 0.0).is_a? Numeric}}

assert_true(numeric_features.size >= 4, "At least 4 numeric features")
def test_ig_numeric extracted_dataset, h0, test_feature1
  t, ig = find_split_point_numeric extracted_dataset, h0, test_feature1
  assert_true(ig >= 0.005, "Expected information gain for '#{test_feature1}' > 0.005")
  return test_feature1
end

test_ig_numeric extracted_dataset, h0, numeric_features[0]

"own_car_age"

In [58]:
test_ig_numeric extracted_dataset, h0, numeric_features[1]

"ext_source_1"

In [59]:
test_ig_numeric extracted_dataset, h0, numeric_features[2]

"ext_source_2"

In [60]:
3.upto(numeric_features.size - 1) do |i|
  test_ig_numeric extracted_dataset, h0, numeric_features[i]
end

3

In [61]:
categorical_features = features.select {|k| extracted_dataset.all? {|row| row["features"].fetch(k, "").is_a? String}}

assert_true(categorical_features.size >= 4, "At least 4 categorical features")

def test_ig_categorical extracted_dataset, h0, test_feature1
  splits = extracted_dataset.group_by {|row| row["features"][test_feature1]}
  ig = information_gain h0, splits
  puts ig
  assert_true(ig >= 0.005, "Expected information gain for '#{test_feature1}' > 0.005")
  return test_feature1
end

test_ig_categorical extracted_dataset, h0, categorical_features[0]

0.021533067921555307


"fondkapremont_mode"

In [62]:
test_ig_categorical extracted_dataset, h0, categorical_features[1]

0.021445287492354463


"housetype_mode"

In [63]:
test_ig_categorical extracted_dataset, h0, categorical_features[2]

0.023309031788923393


"wallsmaterial_mode"

In [64]:
3.upto(categorical_features.size - 1) do |i|
  test_ig_categorical extracted_dataset, h0, categorical_features[i]
end

0.021452835718751083


3